# Converts LES Output from 'NCAR' format to SCAM format

In [1]:
import numpy as np
import metpy.calc as mpc
import metpy.constants as mconst
import xarray as xr
import scipy.ndimage as ndimage
import pandas as pd
import datetime as dt

# My functions
import scam_func as mypy
import importlib

+++ IMPORTING MY FUNCTIONS +++


In [2]:
### SCRIPT TO TRANSLATE DATA FROM SCAM NETCDF FILE TO SAS NETCDF FILE ####

### Constants

p0 = 100.*mconst.pot_temp_ref_press.magnitude # mb->Pa
cp_air = mconst.dry_air_spec_heat_press.magnitude # Specific heat for dry air
r_gas = 1000.*mconst.dry_air_gas_constant.magnitude   # Specific gas constant for dry air
r_cp = r_gas/cp_air    # r/cp
grav = mconst.earth_gravity.magnitude       # Gravity ave.

# Reload scam_func.py
importlib.reload(mypy) # Required because I am constantly editing scam_func.py


###### IOP file(s) info. ######

tlocal_offset_sec = -6*3600 # Hours behind Z (->seconds/hour)

scam_file_nums = ['106d'] # 'a'=L16, ''=L32
scam_run_names = ['CAM6']

scam_file_pre = "history/FSCAM.T42_T42.SAS."
scam_file_suf = ".cam.h0.2013-06-10-39600"

#------- Input/Output Files------------#
scam_files_in = [scam_file_pre+x+scam_file_suf for x in scam_file_nums]
scam_file_out = scam_files_in[0]+'_SAS_format.nc'

### READ IN NETCDF FILE/SET UP OUTPUT FILE ####
scam_in = xr.open_dataset(scam_files_in[0]+'.nc',engine='netcdf4')
scam_in = scam_in.squeeze() # Get rid dim=1 lat and lon

#------------ MakeDataSet copy --------------#
scam_out = scam_in.copy(deep=True)

#------ Dict of data variables ------------#
scam_vars  = scam_in.data_vars 

# Array of all 'data' variable names
scam_vars = list(scam_vars.keys())

#------- Grab SCAM Variable Names -------#
%run scam_nc2sas_vars # Run py file
sas_vars = list(var_r_df.index)

#------------ DROP all SCAM_out data variables  -------------#


scam_out = scam_out.drop_vars(scam_vars)
scam_out = scam_out.drop_vars(['lat','lon']) # Remov x,y coords
#scam_out = scam_out.dtype(float64)


#---------- Straight copy/scale/rename of variables ----------#

print('#------- File Variable Writes ------#')
print('')

#------- Time Manipulation (seconds from loca midnight annoying)---------#

time_sas = scam_in.time.dt.second \
                +60*scam_in.time.dt.minute \
                +3600*scam_in.time.dt.hour \
                +86400*scam_in.time.dt.day



##############################
#---- LOOP ALL VARIABLES ----#
##############################

for vsas in sas_vars:
    vcam = var_r_df.loc[vsas]['SCAM Name']
    vname = var_r_df.loc[vsas]['Long Name']
    
    if vcam != '': # Direct transfer
 
        print('DIRECT  ++ Var (SAS/name/CAM) = ',vsas,' - ',vname,' - ',vcam)
         
        if vsas in ['latitude','longitude']:
            var = xr.DataArray(scam_in[vcam].values) 
        else:        
# Rename (to SAS name) and scale vars to SAS values
            var = scam_in[vcam]*var_r_df.loc[vsas,'Scaling']

    
    #------ Derived Variables ------#

    if vcam == '':   # Derived or other variable
        print('DERVIED ++ Var (SAS/name/CAM) = ',vsas,' - ',vname)
            
# Vertical coords are 2D don't forget.        
        
        if vsas=='p':   pres,zhgt = mypy.vcoord_scam('mid',scam_in)  ;  var=pres
        if vsas=='zu':  pres,zhgt = mypy.vcoord_scam('mid',scam_in)  ;  var=zhgt 
        if vsas=='zw':  
            pint,zint = mypy.vcoord_scam('int',scam_in)  
            var=zint # Interface value that includes the surface value      
            
        
        if vsas in ['t','tv']: ## THETA/THETA_V ##
            pres,zhgt = mypy.vcoord_scam('mid',scam_in)
            temp = scam_in.T 
            var = temp*(p0/pres)**r_cp
            print(p0)
            if vsas=='tv': var = var*(1+0.61*scam_in.T) 
            
## dq/dz / dtheta/dz for PBL depth

        if vsas in ['zi_q','zi_t']:
            dzpbl_v = 'Q' if vsas in 'zi_q' else 'TH' 
            var = mypy.pbl_grad_calc(dzpbl_v,scam_in)

### Last constant variables and time
        if vsas=='time':
            var = time_sas-time_sas[0] # Scale by the first time (NCAR-LES has no time=0, SCAM does)
            
        if vsas in ['ts','cs']:  
            var = time_sas[0]-86400*scam_out.time.dt.day[0]+tlocal_offset_sec         
            var = xr.DataArray(var)
           
   
## Add in general meta data ##            

    var = var.assign_attrs(description = var_r_df.loc[vsas,'Long Name'])
    var = var.assign_attrs(Units = var_r_df.loc[vsas,'Units'])
    
## If an ilev array have to strip out surface value and assign z_loc=2

    
    if 'lev' in var.dims: var = var.assign_attrs(z_loc = '1') # z-level attribute (mid-level)

    if 'ilev' in var.dims:     # Do not include the lowest (surface interface flux) if ilev
        print('- Mapping ilev to lev -')
        var = var.drop_vars({'ilev'}) # Rename ilev to lev.
        var = var[:,0:-1]  # All but bottom interface level.
        var = var.rename({'ilev':'lev'}) # Rename ilev to lev.
        var = var.assign_attrs(z_loc = '2') # Add surface above level attributes 
        
#        
#  Now add dataarray to the output dataset    
#   
   
    print('Min/Max = ',np.amin(var.values),' ',np.amax(var.values))
    var.values = np.float64(var.values)

#    var.attrs['_FillValue'] = -9999.
#    print(var)
    
    scam_out[vsas] = var

    var = None
    
#------ Change Vertical and Time Dimension to to nz/nt ------------#


# Reverse arrays in the vertical first
scam_out = scam_out.reindex(lev=scam_out.lev[::-1])

# Then chnage the dim names
scam_out = scam_out.swap_dims({'time':'nt','lev':'nz'})

# Strip out the coordinates
scam_out = scam_out.drop_vars({'ilev','lev','lat','lon'}) # Remove vertical coordinates
#scam_out = scam_out.squeeze # Remove vestigal vertical coordinate

# Last thing move time to a data coordinate
scam_out = scam_out.reset_coords('time') # Move time to data variable.



print(scam_out)
#---------------- Write Out nc File -----------------#

scam_out.to_netcdf(scam_file_out)

print('-----------')
print('### END ###')
        
## JUNK


#        vsas0 = var_r_df.loc[var_r_df['SCAM Name'] == vcam] # Find index (SAS) variable name
#        vsas = vsas0.index # Get variable as the 'index' 
  
#        vname = var_r_df.loc[vsas]['Long Name']
#        vsas = vsas.map(str).values ; vsas = vsas[0] # Select first value to give single value - ridiculous
#        vname = vname.map(str).values ; vname = vname[0] # Same long-name
    

+++ IMPORTING MY FUNCTIONS +++
#------- File Variable Writes ------#

DERVIED ++ Var (SAS/name/CAM) =  ts  -  Simulation start time
Min/Max =  18000   18000
DERVIED ++ Var (SAS/name/CAM) =  cs  -  Chemistry start time
Min/Max =  18000   18000
DIRECT  ++ Var (SAS/name/CAM) =  latitude  -  Latitude  -  lat
Min/Max =  32.5   32.5
DIRECT  ++ Var (SAS/name/CAM) =  longitude  -  Longitude  -  lon
Min/Max =  272.8499984741211   272.8499984741211
DERVIED ++ Var (SAS/name/CAM) =  zu  -  Height of grid at grid center
Min/Max =  7.777585059669775   29856.681134841125
DERVIED ++ Var (SAS/name/CAM) =  zw  -  Height of grid at grid upper-face
- Mapping ilev to lev -
Min/Max =  15.561037791821605   30053.561655526802
DERVIED ++ Var (SAS/name/CAM) =  p  -  Pressure
Min/Max =  242.87677952088416   96597.59587274864
DIRECT  ++ Var (SAS/name/CAM) =  wqsfc  -  Surface water vapor specific humidity flux  -  LHFLX
Min/Max =  0.0   0.00015
DIRECT  ++ Var (SAS/name/CAM) =  wtsfc  -  Surface potential temperat

# 